In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_data(url, writer, category):
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    items = soup.find_all('div', class_='desc')

    for item in items:
        title_element = item.find("a")
        title = title_element.get("title") if title_element else None

        price_element = item.find('div', class_='price')
        price = price_element.text.strip().replace("Giá:", "") if price_element else ""

        area_element = item.find('div', class_='area')
        area = area_element.text.strip().replace("Diện tích:", "") if area_element else ""

        location_element = item.find('div', class_='location')
        location = location_element.text.strip().replace("Vị trí:", "") if location_element else ""

        date_element = item.find('span', class_='date')
        date = date_element.text.strip() if date_element else ""

        li_element = item.find_parent('li')
        img_tags = li_element.select('img')
        img_src_list = [img.get('src') for img in img_tags if img.get('src')]

        writer.writerow([category, title, price, area, location, date, img_src_list])

# Mở một file CSV để lưu dữ liệu
with open('nhadat.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Loại', 'Tiêu đề', 'Giá', 'Diện tích', 'Vị trí', 'Ngày đăng', 'Danh sách đường dẫn'])

    for page in range(1, 1000):
        url_ban = f"https://dothi.net/nha-dat-ban/p{page}.htm"
        scrape_data(url_ban, writer, "Nhà đất bán")

        url_cho_thue = f"https://dothi.net/nha-dat-cho-thue/p{page}.htm"
        scrape_data(url_cho_thue, writer, "Nhà đất cho thuê")

print("Scraping completed and data saved to nhadat.csv")
